<a href="https://colab.research.google.com/github/mridulbiswas402/entropy-based-stegnography/blob/main/finalstego.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
from math import log
from math import e
from tqdm import tqdm
from math import log10, sqrt
from scipy import stats
#from PIL import ImageColor
#from PIL import Image, ImageFilter
import binascii
import cv2 
from numpy import asarray
from math import floor
import numpy as np 
from google.colab.patches import cv2_imshow  
import matplotlib.pyplot as plt 
%matplotlib inline
import os
import pandas as pd

In [47]:
def result(imgdata,binary,mask):
  entry = len(imgdata)
  score = list()
  for i in tqdm(range(entry)):
    image = cv2.imread(imgdata["path"][i]) 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    temp=gray.copy()
    temp=varbitstego.varbitencode1(binary,temp,mask)
    score.append(varbitstego.metrics(gray,temp))
  colnames=["BPP","MSE","SNR","PSNR","DATA"]  
  return  pd.DataFrame(score,columns=colnames)



def metrics(original,output,imageid):
    bpp = BPP(original)
    mse = MSE(original,output)
    snr = SNR(original,output)
    psnr = PSNR(original,output)
    noofpix = len(edgeloc(original))
    dat = (bpp*noofpix)/8
    return imageid,bpp,mse,snr,psnr,dat


def MSE(output,input):
    difference_array = np.subtract(output, input)
    squared_array = np.square(difference_array)
    mse = squared_array.mean()
    return mse

def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr  

def SNR(original, compressed):
    mse = np.sum((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = np.sum(np.square(original))
    snr = 20 * log10(max_pixel / mse)
    return snr

def BPP(grayimagearray):
    edgpix = edgepixval(grayimagearray)
    noofbit = floor(log(np.std(edgpix),e))
    return noofbit     

In [48]:
def edgeloc(imagearray):
    edges = cv2.Canny(imagearray,100,200)
    row,col=edges.shape
    lst=[]
    for i in range(1,row):
      for j in range(col):
        if(edges[i,j]>=250):
          lst.append((i,j))  
    return lst 

def edgepixval(imagearray):
    edgloc=edgeloc(imagearray)
    edglst=[]
    l = len(edgloc)
    for k in range(l):
      i,j=edgloc[k]
      edglst.append(imagearray[i,j])
    return np.array(edglst)  


def calcmask(grayimagearray):
    edgpix = edgepixval(grayimagearray)
    noofbit = floor(log(np.std(edgpix),e))
    mask = ''
    for i in range(8-noofbit):
      mask=mask+'1'
    for i in range(noofbit):
      mask = mask + '0'  
    return int(mask,2)  

In [49]:
def rgb2hex(r, g, b):
	return '#{:02x}{:02x}{:02x}'.format(r, g, b)

def hex2rgb(hexcode):
	return ImageColor.getcolor(hexcode, "RGB")


def str2bin(message):
    binary = bin(int(binascii.hexlify(message.encode("ascii")), 16))
    return binary[2:]  

def bin2str(binary):
	message = binascii.unhexlify('%x' % (int('0b'+binary,2)))
	return message


In [50]:
def varbitextr(pixel,mask):
    noofbits=countzeros(mask)
    dmask=mask ^ 255
    if (noofbits==1):
      return format((pixel & dmask),"0b")
    elif (noofbits==2):
      return format((pixel & dmask),"02b")
    elif (noofbits==3):
      return format((pixel & dmask),"03b") 
    elif (noofbits==4):
      return format((pixel & dmask),"04b")
    else:
      print("error")     


def varbitemb(pixel,data,mask):
  return ((pixel & mask )|data)



def countzeros(mask):
  count0=0
  while (mask != 0):  
    if (mask % 2 == 0):
      count0 += 1
    mask //= 2
  return count0  


def stuffbits(binarymessage,mask):
 # print("  in function stuffbit ")
  n = countzeros(mask)
  #print("on of zero in mask = ",n)
  nbstuff = (n-(len(binarymessage) % n)) 
  #print("nbstuff = ",nbstuff)
  while (nbstuff):
    binarymessage=binarymessage+'0'
    nbstuff=nbstuff-1
  return binarymessage   


In [51]:
def varbitencode1(binmessage,imagearray,mask):
 # print("in function varbitencode1 ")
  row,col=imagearray.shape
 # print("row,col = ",row,col)
  noofbits= countzeros(mask)
 # print("noofbit = ",noofbits)
  lst = stuffbits(binmessage,mask)
  #lst=binmessage
  imagearray[0,0]=mask
  noofpix = format(int((len(lst)/noofbits)),'016b')
 # print("noofpix = ",int(noofpix,2))
  datalen = format(len(binmessage),'016b')
 # print("datalen = ",int(datalen,2))
  imagearray[0,1]=int(noofpix[:8],2)
  imagearray[0,2]=int(noofpix[8:],2)
  imagearray[0,3]=int(datalen[:8],2)
  imagearray[0,4]=int(datalen[8:],2)
  for i in range(1,row):
    for j in range(col):
      temp = imagearray[i,j]
      if (lst):
        data=int(lst[:noofbits],2)
        lst=lst[noofbits:]
        imagearray[i,j]=varbitemb(temp,data,mask)
      else: break
  return imagearray 


 
def varbitdecode1(imagearray):
 # print("  in function varbitdecode1  ")
  #dmask = mask ^ 255
  mask = imagearray[0,0]
 # print("mask = ",mask)
  noofpix = (256*imagearray[0,1]+imagearray[0,2])
 # print("noofpix = ",noofpix)
  datalen = (256*imagearray[0,3]+imagearray[0,4])
 # print("datalen = ",datalen)
  binmessage=''
  #noofbits= countzeros(mask)
  n = countzeros(mask)
 # print("no of zeros in mask = ",n)
  stuffedbit = (n-(datalen % n))
 # print("stuffedbit = ",stuffedbit) 
  row,col=imagearray.shape
 # print("row,col = ",row,col)

  for i in range(1,row):
    for j in range(col):
      temp = imagearray[i,j]
      binmessage=binmessage+varbitextr(temp,mask)
      noofpix=noofpix-1
      if(not noofpix):
        print('success')
        return bin2str(binmessage[:-stuffedbit])
       # print(-stuffedbit)
        #return binmessage[:-stuffedbit] to

In [52]:

def varbitencode2(binmessage,imagearray,mask):
 # print("in function varbitencode1 ")
  row,col=imagearray.shape
 # print("row,col = ",row,col)
  noofbits= countzeros(mask)
 # print("noofbit = ",noofbits)
  lst = stuffbits(binmessage,mask)
  #lst=binmessage
  imagearray[0,0]=mask
  noofpix = format(int((len(lst)/noofbits)),'016b')
 # print("noofpix = ",int(noofpix,2))
  datalen = format(len(binmessage),'016b')
 # print("datalen = ",int(datalen,2))
  imagearray[0,1]=int(noofpix[:8],2)
  imagearray[0,2]=int(noofpix[8:],2)
  imagearray[0,3]=int(datalen[:8],2)
  imagearray[0,4]=int(datalen[8:],2)
 ###################################
  edg = edgeloc(imagearray)
  nofedg = len(edg)
  for k in range(nofedg):
      i,j= edg[k]
      temp = imagearray[i,j]
      if (lst):
        data=int(lst[:noofbits],2)
        lst=lst[noofbits:]
        imagearray[i,j]=varbitemb(temp,data,mask)
      else: break
  return imagearray 

"""
# correction needed
def varbitdecode2(imagearray):
 # print("  in function varbitdecode1  ")
  #dmask = mask ^ 255
  mask = imagearray[0,0]
 # print("mask = ",mask)
  noofpix = (256*imagearray[0,1]+imagearray[0,2])
 # print("noofpix = ",noofpix)
  datalen = (256*imagearray[0,3]+imagearray[0,4])
 # print("datalen = ",datalen)
  binmessage=''
  #noofbits= countzeros(mask)
  n = countzeros(mask)
 # print("no of zeros in mask = ",n)
  stuffedbit = (n-(datalen % n))
 # print("stuffedbit = ",stuffedbit) 
  row,col=imagearray.shape
 # print("row,col = ",row,col)
  edg = edgeloc(imagearray)
  nofedg = len(edg)
  for k in range(nofedg):
      i,j= edg[k]
      temp = imagearray[i,j]
      binmessage=binmessage+varbitextr(temp,mask)
      noofpix=noofpix-1
      if(not noofpix):
        print('success')
        return bin2str(binmessage[:-stuffedbit])
       # print(-stuffedbit)
        #return binmessage[:-stuffedbit] to  

  """      

'\n# correction needed\ndef varbitdecode2(imagearray):\n # print("  in function varbitdecode1  ")\n  #dmask = mask ^ 255\n  mask = imagearray[0,0]\n # print("mask = ",mask)\n  noofpix = (256*imagearray[0,1]+imagearray[0,2])\n # print("noofpix = ",noofpix)\n  datalen = (256*imagearray[0,3]+imagearray[0,4])\n # print("datalen = ",datalen)\n  binmessage=\'\'\n  #noofbits= countzeros(mask)\n  n = countzeros(mask)\n # print("no of zeros in mask = ",n)\n  stuffedbit = (n-(datalen % n))\n # print("stuffedbit = ",stuffedbit) \n  row,col=imagearray.shape\n # print("row,col = ",row,col)\n  edg = edgeloc(imagearray)\n  nofedg = len(edg)\n  for k in range(nofedg):\n      i,j= edg[k]\n      temp = imagearray[i,j]\n      binmessage=binmessage+varbitextr(temp,mask)\n      noofpix=noofpix-1\n      if(not noofpix):\n        print(\'success\')\n        return bin2str(binmessage[:-stuffedbit])\n       # print(-stuffedbit)\n        #return binmessage[:-stuffedbit] to  \n\n  '

In [53]:
def statistics(imgdata,binary,encodefunc):     # imgdata is dataframe
  entry = len(imgdata)
  score = list()
  for i in tqdm(range(entry)):
    image = cv2.imread(imgdata["path"][i]) 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    temp=gray.copy()
    mask = calcmask(gray)
    temp= encodefunc(binary,temp,mask)
    score.append(metrics(gray,temp,imgdata["imageid"][i]))
  colnames=["Image_ID","BPP","MSE","SNR","PSNR","DATA"]  
  return  pd.DataFrame(score,columns=colnames)




In [54]:
###########################################################################

In [55]:
base_image_dir ='/content/drive/MyDrive/imgdata'
df = pd.read_csv('imagedat.csv')
df['path'] = df['imageid'].map(lambda x: os.path.join(base_image_dir,'{}.tif'.format(x)))
#df = df.drop(columns=['id_code'])

In [56]:
file = open('hello.txt', encoding='ascii', errors='replace')
#print(file.read())
message = file.read()
file.close()

In [57]:
#message

In [58]:
binary= str2bin(message)
               

In [59]:
#print(bin2str(binary))

In [60]:
stats=statistics(df,binary,varbitencode1)

100%|██████████| 10/10 [00:13<00:00,  1.37s/it]


In [61]:
stats

,Image_ID,BPP,MSE,SNR,PSNR,DATA
0,1,4,0.102589,60.771366,58.019809,2019.500
1,2,3,0.007845,82.463710,69.184761,2665.500
2,3,4,0.072957,54.130157,59.500110,3045.000
3,4,3,0.011990,79.906692,67.342496,846.750
4,5,3,0.013756,78.495429,66.745946,6431.625
5,6,3,0.012791,78.353574,67.061867,5831.625
6,7,3,0.012463,78.994797,67.174711,5506.125
7,8,3,0.007355,81.983484,69.465133,2247.750
8,9,3,0.012398,79.334892,67.197369,10495.125
9,10,3,0.012424,78.830443,67.188025,20851.125


In [62]:
################################

In [63]:
stats=statistics(df,binary,varbitencode2)

100%|██████████| 10/10 [00:17<00:00,  1.77s/it]


In [64]:
stats

,Image_ID,BPP,MSE,SNR,PSNR,DATA
0,1,4,0.111963,60.011838,57.640045,2019.500
1,2,3,0.012973,78.095316,67.000564,2665.500
2,3,4,0.035686,60.341611,62.605837,3045.000
3,4,3,0.011932,79.948908,67.363604,846.750
4,5,3,0.012482,79.339677,67.168070,6431.625
5,6,3,0.013561,77.845453,66.807807,5831.625
6,7,3,0.011051,80.038810,67.696718,5506.125
7,8,3,0.012817,77.158839,67.052810,2247.750
8,9,3,0.012218,79.461420,67.260633,10495.125
9,10,3,0.012814,78.562598,67.054103,20851.125
